<a href="https://colab.research.google.com/github/lindaelmi/Tasks/blob/main/task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# 1. Installe Yargy sans DAWG
!pip install yargy

# 2. Installe pymorphy2 **sans la partie fast** (évite DAWG)
!pip install pymorphy2 dawg-python

# 3. Vérifie
import yargy, pymorphy2
print("✅ Yargy + pymorphy2 OK")

  Using cached yargy-0.16.0-py3-none-any.whl.metadata (3.5 kB)
  Using cached pymorphy2-0.9.1-py3-none-any.whl.metadata (3.6 kB)
  Using cached DAWG_Python-0.7.2-py2.py3-none-any.whl.metadata (7.0 kB)
  Using cached pymorphy2_dicts_ru-2.4.417127.4579844-py2.py3-none-any.whl.metadata (2.1 kB)
  Using cached docopt-0.6.2-py2.py3-none-any.whl
Using cached yargy-0.16.0-py3-none-any.whl (33 kB)
Using cached pymorphy2-0.9.1-py3-none-any.whl (55 kB)
Using cached DAWG_Python-0.7.2-py2.py3-none-any.whl (11 kB)
Using cached pymorphy2_dicts_ru-2.4.417127.4579844-py2.py3-none-any.whl (8.2 MB)
✅ Yargy + pymorphy2 OK


In [7]:
# 3. Импортируем все необходимые модули для работы
from yargy import Parser, rule, and_, not_
from yargy.interpretation import fact
from yargy.predicates import gram
from yargy.relations import gnc_relation

In [8]:
# 4. Загружаем файл news.txt с компьютера в Colab
from google.colab import files
uploaded = files.upload()   # sélectionne news.txt
# le fichier est maintenant dans /content/news.txt

Saving news.txt to news.txt


In [11]:
import pathlib, json, re
from dataclasses import dataclass
from typing import Optional, List
# 5. Указываем пути к входному и выходному файлам
FILE_PATH   = pathlib.Path("/content/news.txt")   # fichier que tu viens d’uploader
OUTPUT_PATH = pathlib.Path("/content/entries.json")
# 6. Определяем структуры для извлечения имён и персон
# structures Yargy
Name = fact('Name', ['first', 'last'])
Person = fact('Person', ['name', 'birth_date', 'birth_place'])

In [13]:
# ---------- Grammaire : Nom ----------
# 7. Создаём грамматику для поиска Имя + Фамилия
# gnc_relation обеспечивает согласование по падежу, числу, роду

gnc = gnc_relation()
FIRST = and_(gram('Name'), not_(gram('Abbr'))).interpretation(Name.first).match(gnc)
LAST  = and_(gram('Surn'), not_(gram('Abbr'))).interpretation(Name.last).match(gnc)
NAME  = rule(FIRST, LAST).interpretation(Name)
PERSON_NAME = rule(NAME.interpretation(Person.name)).interpretation(Person)
parser = Parser(PERSON_NAME)
# 8. Регулярное выражение для поиска дат рождения вида "22 ноября 1987"
# ---------- Dates ----------
DATE_PATTERN = re.compile(r'\b([1-3]?\d)\s(января|февраля|марта|апреля|мая|июня|июля|августа|сентября|октября|ноября|декабря)\s(\d{4})\b', re.IGNORECASE)

# ---------- Lieux ----------
# 9. Список городов для извлечения места рождения
CITIES = ['Москва', 'Санкт-Петербург', 'Казань', 'Минск', 'Генуя', 'Париж', 'Лондон', 'Киев', 'Симферополь', 'Нью-Йорк', 'Рим', 'Варшава', 'Берлин']
def extract_place(text: str) -> Optional[str]:
    for city in CITIES:
        if re.search(rf'\b(?:в|на)\s{city}\b', text):
            return city
    return None

# ---------- Prénoms ----------
# 10. Список русских имён для фильтрации ложных срабатываний
RUSSIAN_FIRST_NAMES = {
    'Александр', 'Алексей', 'Андрей', 'Антон', 'Артём', 'Борис', 'Вадим', 'Валентин', 'Василий', 'Виктор', 'Владимир', 'Владислав', 'Геннадий', 'Георгий', 'Даниил', 'Денис', 'Дмитрий', 'Евгений', 'Иван', 'Игорь', 'Илья', 'Кирилл', 'Константин', 'Леонид', 'Максим', 'Михаил', 'Никита', 'Николай', 'Олег', 'Павел', 'Пётр', 'Роман', 'Сергей', 'Станислав', 'Степан', 'Тимур', 'Фёдор', 'Юрий', 'Ярослав',
    'Анна', 'Валентина', 'Вера', 'Виктория', 'Галина', 'Дарья', 'Екатерина', 'Елена', 'Зоя', 'Ирина', 'Ксения', 'Лариса', 'Людмила', 'Маргарита', 'Мария', 'Наталья', 'Оксана', 'Ольга', 'Светлана', 'Софья', 'Татьяна', 'Юлия', 'Яна'
}
def is_likely_person(name: str) -> bool:
    parts = name.split()
    if len(parts) != 2: return False
    first, last = parts
    return (first in RUSSIAN_FIRST_NAMES) or (last in RUSSIAN_FIRST_NAMES)

In [14]:
# 11. Обрабатываем файл news.txt построчно
all_results = []

with open(FILE_PATH, "r", encoding="utf-8") as f:
    for line in f:
        parts = line.strip().split("\t")
        if len(parts) != 3: continue
        category, title, text = parts

        for match in parser.findall(text):
            p = match.fact
            if p.name and is_likely_person(f"{p.name.first} {p.name.last}"):
                date_match = re.search(DATE_PATTERN, text)
                place_match = extract_place(text)
                entry = {
                    "name": f"{p.name.first} {p.name.last}",
                    "birth_date": date_match.group(0) if date_match else None,
                    "birth_place": place_match
                }
                all_results.append(entry)

print(f"✅ Extracted {len(all_results)} entries")

✅ Extracted 4659 entries


In [15]:
# 12. Сохраняем результат в JSON и скачиваем на компьютер
with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
    json.dump(all_results, f, ensure_ascii=False, indent=2)

from google.colab import files
files.download(OUTPUT_PATH)  # скачать файл
print("✅ Файл entries.json сохранён и скачан")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Файл entries.json сохранён и скачан
